In [9]:
text = map(str, ['hi', 5, 'bye'])
print('Output of MultiDCP:{}'.format("\n".join(text)))

NameError: name 'apply' is not defined

In [2]:
import os
import pandas as pd
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import sys
from datetime import datetime
import torch
from torch import save
import numpy as np
import argparse
sys.path.insert(0, os.path.abspath('../module-subdirectory'))
sys.path.append(os.path.dirname(os.path.abspath('')) + '/models')
sys.path.append(os.path.dirname(os.path.abspath('')) + '/utils')
import multidcp
import datareader
import metric
import wandb
import pdb
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from multidcp_ae_utils import *

ModuleNotFoundError: No module named 'multidcp'

In [7]:
!pwd

/raid/home/joshua/MultiDCP/MultiDCP


In [9]:
os.getcwd()

'/raid/home/joshua/MultiDCP/MultiDCP'

In [6]:
 print(os.path.abspath('../models'))

/raid/home/joshua/MultiDCP/models


In [12]:
import os
import pandas as pd
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import sys
from datetime import datetime
import torch
from torch import save
import numpy as np
import argparse
sys.path.insert(0, os.path.abspath('./models'))
sys.path.insert(0, os.path.abspath('./utils'))
import multidcp
import datareader
import metric
import wandb
import pdb
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from multidcp_ae_utils import *


In [13]:

USE_WANDB = True
if USE_WANDB:
    wandb.init(project="MultiDCP_AE_loss")
else:
    os.environ["WANDB_MODE"] = "dryrun"

# check cuda
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Use GPU: %s" % torch.cuda.is_available())

def model_training(args, model, data, ae_data, metrics_summary):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)
    best_dev_pearson = float("-inf")

    for epoch in range(args.max_epoch):
    
        print("Iteration %d:" % (epoch))
        print_lr(optimizer)
        model.train()
        data_save = False
        epoch_loss = 0

        for i, (feature, label, _) in enumerate(ae_data.train_dataloader()):

            optimizer.zero_grad()
            #### the auto encoder step doesn't need other input rather than feature
            predict, cell_hidden_ = model(input_cell_gex=feature, job_id = 'ae', epoch = epoch)
            loss_t = model.loss(label, predict)
            loss_t.backward()
            optimizer.step()
            epoch_loss += loss_t.item()

        print('AE Train loss:')
        print(epoch_loss/(i+1))
        if USE_WANDB:
            wandb.log({'AE Train loss': epoch_loss/(i+1)}, step = epoch)

        model.eval()
        epoch_loss = 0
        lb_np = np.empty([0, 978])
        predict_np = np.empty([0, 978])
        with torch.no_grad():
            for i, (feature, label, _) in enumerate(ae_data.val_dataloader()):
                predict, _ = model(input_cell_gex=feature, job_id = 'ae', epoch = epoch)
                loss = model.loss(label, predict)
                epoch_loss += loss.item()
                lb_np = np.concatenate((lb_np, label.cpu().numpy()), axis=0)
                predict_np = np.concatenate((predict_np, predict.cpu().numpy()), axis=0)
            validation_epoch_end(epoch_loss = epoch_loss, lb_np = lb_np, 
                                predict_np = predict_np, steps_per_epoch = i+1, 
                                epoch = epoch, metrics_summary = metrics_summary,
                                job = 'ae', USE_WANDB = USE_WANDB)
        
        model.train()
        epoch_loss = 0
        for i, (ft, lb, _) in enumerate(data.train_dataloader()):
            drug = ft['drug']
            mask = ft['mask']
            cell_feature = ft['cell_id']
            pert_idose = ft['pert_idose']
            optimizer.zero_grad()
            predict, cell_hidden_ = model(input_cell_gex=cell_feature, input_drug = drug, 
                                        input_gene = data.gene, mask = mask,
                                        input_pert_idose = pert_idose, 
                                        job_id = 'perturbed', epoch = epoch)
            loss_t = model.loss(lb, predict)
            loss_t.backward()
            optimizer.step()
            if i == 1:
                print('__________________________pertubed input__________________________')
                print(cell_feature)
                print('__________________________pertubed hidden__________________________')
                print(cell_hidden_)
                print('__________________________pertubed predicts__________________________')
                print(cell_hidden_)
            epoch_loss += loss_t.item()
        print('Perturbed gene expression profile Train loss:')
        print(epoch_loss/(i+1))
        if USE_WANDB:
            wandb.log({'Perturbed gene expression profile Train loss': epoch_loss/(i+1)}, step = epoch)

        model.eval()
        epoch_loss = 0
        lb_np = np.empty([0, 978])
        predict_np = np.empty([0, 978])
        with torch.no_grad():
            for i, (ft, lb, _) in enumerate(data.val_dataloader()):
                drug = ft['drug']
                mask = ft['mask']
                cell_feature = ft['cell_id']
                pert_idose = ft['pert_idose']
                predict, _ = model(input_cell_gex=cell_feature, input_drug = drug, 
                                input_gene = data.gene, mask = mask,
                                input_pert_idose = pert_idose, 
                                job_id = 'perturbed', epoch = epoch)
                loss = model.loss(lb, predict)
                epoch_loss += loss.item()
                lb_np = np.concatenate((lb_np, lb.cpu().numpy()), axis=0)
                predict_np = np.concatenate((predict_np, predict.cpu().numpy()), axis=0)
            validation_epoch_end(epoch_loss = epoch_loss, lb_np = lb_np, 
                                predict_np = predict_np, steps_per_epoch = i+1, 
                                epoch = epoch, metrics_summary = metrics_summary,
                                job = 'perturbed', USE_WANDB = USE_WANDB)

            if best_dev_pearson < metrics_summary['pearson_list_perturbed_dev'][-1] or epoch == 1:
                # data_save = True
                best_dev_pearson = metrics_summary['pearson_list_perturbed_dev'][-1]
                torch.save(model.state_dict(), 'best_multidcp_ae_model_1.pt')
        # if not data_save or (epoch < 400 and epoch != 1):
        #     continue
        epoch_loss = 0
        lb_np = np.empty([0, 978])
        predict_np = np.empty([0, 978])
        hidden_np = np.empty([0, 50])
        with torch.no_grad():
            for i, (feature, label, _) in enumerate(ae_data.test_dataloader()):
                predict, hidden = model(input_cell_gex=feature, job_id = 'ae')
                loss = model.loss(label, predict)
                epoch_loss += loss.item()
                lb_np = np.concatenate((lb_np, label.cpu().numpy()), axis=0)
                predict_np = np.concatenate((predict_np, predict.cpu().numpy()), axis=0)
                hidden_np = np.concatenate((hidden_np, hidden.cpu().numpy()), axis=0)

            if data_save:
                test_ae_label_file = pd.read_csv(args.ae_label_file + '_test.csv', index_col=0)
                hidden_df = pd.DataFrame(hidden_np, index = list(test_ae_label_file.index), columns = [x for x in range(50)])
                print('++++++++++++++++++++++++++++Write hidden state out++++++++++++++++++++++++++++++++')
                hidden_df.to_csv(args.hidden_repr_result_for_testset)

            test_epoch_end(epoch_loss = epoch_loss, lb_np = lb_np, 
                                predict_np = predict_np, steps_per_epoch = i+1, 
                                epoch = epoch, metrics_summary = metrics_summary,
                                job = 'ae', USE_WANDB = USE_WANDB)

        epoch_loss = 0
        lb_np_ls = []
        predict_np_ls = []
        hidden_np_ls = []
        with torch.no_grad():
            for i, (ft, lb, _) in enumerate(tqdm(data.test_dataloader())):
                drug = ft['drug']
                mask = ft['mask']
                cell_feature = ft['cell_id']
                pert_idose = ft['pert_idose']
                predict, cells_hidden_repr = model(input_cell_gex=cell_feature, input_drug = drug, 
                                                input_gene = data.gene, mask = mask,
                                                input_pert_idose = pert_idose, job_id = 'perturbed')
                loss = model.loss(lb, predict)
                epoch_loss += loss.item()
                lb_np_ls.append(lb.cpu().numpy()) 
                predict_np_ls.append(predict.cpu().numpy()) 
                hidden_np_ls.append(cells_hidden_repr.cpu().numpy()) 

            lb_np = np.concatenate(lb_np_ls, axis = 0)
            predict_np = np.concatenate(predict_np_ls, axis = 0)
            hidden_np = np.concatenate(hidden_np_ls, axis = 0)
            if data_save:
                sorted_test_input = pd.read_csv(args.test_file).sort_values(['pert_id', 'pert_type', 'cell_feature', 'pert_idose'])
                genes_cols = sorted_test_input.columns[5:]
                assert sorted_test_input.shape[0] == predict_np.shape[0]
                predict_df = pd.DataFrame(predict_np, index = sorted_test_input.index, columns = genes_cols)
                ground_truth_df = pd.DataFrame(lb_np, index = sorted_test_input.index, columns = genes_cols)
                result_df = pd.concat([sorted_test_input.iloc[:, :5], predict_df], axis = 1)
                ground_truth_df = pd.concat([sorted_test_input.iloc[:,:5], ground_truth_df], axis = 1)

                print("=====================================write out data=====================================")
                if epoch == 1:
                    result_df.loc[[x for x in range(len(result_df)//100)],:].to_csv('../MultiDCP/data/teacher_student/second_AD_dataset_results.csv', index = False)
                    # hidden_df.loc[[x for x in range(len(hidden_df))],:].to_csv('../MultiDCP/data/AMPAD_data/second_AD_dataset_hidden_representation.csv', index = False)
                else:
                    result_df.loc[[x for x in range(len(result_df))],:].to_csv(args.predicted_result_for_testset, index = False)
                # hidden_df.loc[[x for x in range(len(hidden_df))],:].to_csv(args.hidden_repr_result_for_testset, index = False)
                # ground_truth_df.loc[[x for x in range(len(result_df))],:].to_csv('../MultiDCP/data/side_effect/test_for_same.csv', index = False)

            test_epoch_end(epoch_loss = epoch_loss, lb_np = lb_np, 
                                predict_np = predict_np, steps_per_epoch = i+1, 
                                epoch = epoch, metrics_summary = metrics_summary,
                                job = 'perturbed', USE_WANDB = USE_WANDB)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joshroll (use `wandb login --relogin` to force relogin)


Use GPU: False


In [19]:

parser = argparse.ArgumentParser(description='MultiDCP AE')

parser.add_argument('--drug_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/all_drugs_l1000.csv")
parser.add_argument('--gene_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/gene_vector.csv")
parser.add_argument('--train_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/pert_transcriptom/signature_train_cell_1.csv")
parser.add_argument('--dev_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/pert_transcriptom/signature_dev_cell_1.csv")
parser.add_argument('--test_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/pert_transcriptom/signature_test_cell_1.csv")
parser.add_argument('--batch_size', type = int, default=64)
parser.add_argument('--ae_input_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/gene_expression_for_ae/gene_expression_combat_norm_978_split4")
parser.add_argument('--ae_label_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/gene_expression_for_ae/gene_expression_combat_norm_978_split4")
parser.add_argument('--cell_ge_file', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/adjusted_ccle_tcga_ad_tpm_log2.csv", help='the file which used to map cell line to gene expression file')
parser.add_argument('--max_epoch', type = int, default=3) # default is 500
parser.add_argument('--predicted_result_for_testset', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/teacher_student/teach_stu_perturbedGX.csv", help = "the file directory to save the predicted test dataframe")
parser.add_argument('--hidden_repr_result_for_testset', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/teacher_student/teach_stu_perturbedGX_hidden.csv", help = "the file directory to save the test data hidden representation dataframe")
parser.add_argument('--all_cells', type=str, default="/raid/home/joshua/MultiDCP/MultiDCP/data/ccle_tcga_ad_cells.p")
parser.add_argument('--dropout', type=float, default=0.3)
parser.add_argument('--linear_encoder_flag', dest = 'linear_encoder_flag', action='store_true', default=False, \
                    help = 'whether the cell embedding layer only have linear layers')

args, unknown = parser.parse_known_args()

all_cells = list(pickle.load(open(args.all_cells, 'rb')))
DATA_FILTER = {"time": "24H", "pert_id": ['BRD-U41416256', 'BRD-U60236422','BRD-U01690642','BRD-U08759356','BRD-U25771771', 'BRD-U33728988', 'BRD-U37049823',
            'BRD-U44618005', 'BRD-U44700465','BRD-U51951544', 'BRD-U66370498','BRD-U68942961', 'BRD-U73238814',
            'BRD-U82589721','BRD-U86922168','BRD-U97083655'],
        "pert_type": ["trt_cp"],
        "cell_id": all_cells,# ['A549', 'MCF7', 'HCC515', 'HEPG2', 'HS578T', 'PC3', 'SKBR3', 'MDAMB231', 'JURKAT', 'A375', 'BT20', 'HELA', 'HT29', 'HA1E', 'YAPC'],
        "pert_idose": ["0.04 um", "0.12 um", "0.37 um", "1.11 um", "3.33 um", "10.0 um"]}



In [21]:

ae_data = datareader.AEDataLoader(device, args)
data = datareader.PerturbedDataLoader(DATA_FILTER, device, args)
ae_data.setup()
data.setup()
print('#Train: %d' % len(data.train_data))
print('#Dev: %d' % len(data.dev_data))
print('#Test: %d' % len(data.test_data))
print('#Train AE: %d' % len(ae_data.train_data))
print('#Dev AE: %d' % len(ae_data.dev_data))
print('#Test AE: %d' % len(ae_data.test_data))

# parameters initialization
model_param_registry = initialize_model_registry()
model_param_registry.update({'num_gene': np.shape(data.gene)[0],
                            'pert_idose_input_dim': len(DATA_FILTER['pert_idose']),
                            'dropout': args.dropout, 
                            'linear_encoder_flag': args.linear_encoder_flag}) 

# model creation
print('--------------with linear encoder: {0!r}--------------'.format(args.linear_encoder_flag))
model = multidcp.MultiDCP_AE(device=device, model_param_registry=model_param_registry)
model.init_weights(pretrained = False)
model.to(device)
model = model.double()     

if USE_WANDB:
    wandb.watch(model, log="all")

# training
metrics_summary = defaultdict(
    pearson_list_ae_dev = [],
    pearson_list_ae_test = [],
    pearson_list_perturbed_dev = [],
    pearson_list_perturbed_test = [],
    spearman_list_ae_dev = [],
    spearman_list_ae_test = [],
    spearman_list_perturbed_dev = [],
    spearman_list_perturbed_test = [],
    rmse_list_ae_dev = [],
    rmse_list_ae_test = [],
    rmse_list_perturbed_dev = [],
    rmse_list_perturbed_test = [],
    precisionk_list_ae_dev = [],
    precisionk_list_ae_test = [],
    precisionk_list_perturbed_dev = [],
    precisionk_list_perturbed_test = [],
)

model_training(args, model, data, ae_data, metrics_summary)
report_final_results(metrics_summary, ae = True, perturbed = True)
end_time = datetime.now()
#torch.save(model, './savedmodel_06232022.pt')
pickle.dump(model, open('model_06232022_ipynb.pkl','wb'))


Feature Summary (printing from data_utils):
['trt_cp']
['A375', 'BT20', 'HA1E', 'HCC515', 'HEPG2', 'HS578T', 'HT29', 'JURKAT', 'PC3', 'YAPC']
['0.04 um', '0.12 um', '0.37 um', '1.11 um', '10.0 um', '3.33 um']
Feature Summary (printing from data_utils):
['trt_cp']
['MCF7', 'SKBR3']
['0.04 um', '0.12 um', '0.37 um', '1.11 um', '10.0 um', '3.33 um']
Feature Summary (printing from data_utils):
['trt_cp']
['A549', 'HELA', 'MDAMB231']
['0.04 um', '0.12 um', '0.37 um', '1.11 um', '10.0 um', '3.33 um']
#Train: 2641
#Dev: 450
#Test: 503
#Train AE: 13053
#Dev AE: 923
#Test AE: 13053
--------------with linear encoder: False--------------
Initialized multidcp's weight............
Iteration 0:
============current learning rate is 0.0002
AE Train loss:
1.8564765752826364
ae Dev loss:
0.6604237909347733
ae RMSE: 0.8122462571051241
ae Pearson's correlation: 0.20437006351796647
ae Spearman's correlation: 0.2226982444748925
ae Precision@10 Positive: 0.1783315276273023
ae Precision@10 Negative: 0.1856988

100%|██████████| 8/8 [00:27<00:00,  3.43s/it]


perturbed Test loss:
3.4031674320182934
perturbed RMSE: 1.8452449119152388
perturbed Pearson's correlation: 0.1654790894245767
perturbed Spearman's correlation: 0.158594388344632
perturbed Precision@10 Positive: 0.14731610337972167
perturbed Precision@10 Negative: 0.1996023856858847
perturbed Precision@20 Positive: 0.1466202783300199
perturbed Precision@20 Negative: 0.21202783300198808
perturbed Precision@50 Positive: 0.12656063618290259
perturbed Precision@50 Negative: 0.18990059642147122
perturbed Precision@100 Positive: 0.11296222664015905
perturbed Precision@100 Negative: 0.16836978131212724
Iteration 1:
============current learning rate is 0.0002
AE Train loss:
1.5547581116921456
ae Dev loss:
0.5944412588000354
ae RMSE: 0.7713868982042773
ae Pearson's correlation: 0.3903308332579248
ae Spearman's correlation: 0.3945275050466697
ae Precision@10 Positive: 0.3872156013001083
ae Precision@10 Negative: 0.32979414951245933
ae Precision@20 Positive: 0.3598049837486457
ae Precision@20 Neg

KeyboardInterrupt: 

In [ ]:
pickle.dump(model, open('model_06232022_ipynb.pkl','wb'))
